# Linear regression performance benchmark: scikit-learn vs. statsmodels

Wu Sun

2019-05-03

This notebook tests the performance of running ordinary least square (OLS) linear regression on samples of an intermediate size (*n* ~ 10<sup>2</sup>–10<sup>4</sup>) using [Scikit-learn](https://scikit-learn.org/stable/) vs. using [Statsmodels](https://www.statsmodels.org/stable/index.html).

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr
from sklearn import datasets
from sklearn.linear_model import LinearRegression

## Version information

In [2]:
import pkg_resources

for pkg in ["numpy", "scipy", "pandas", "statsmodels", "scikit-learn"]:
    version = pkg_resources.get_distribution(pkg).version
    print(f"{pkg} version = {version}")

numpy version = 1.16.3
scipy version = 1.2.1
pandas version = 0.24.2
statsmodels version = 0.9.0
scikit-learn version = 0.19.2


## Load the dataset

Load the Boston house prices dataset from Scikit-learn.

In [3]:
dataset = datasets.load_boston()
X = dataset.data
y = dataset.target
# add constants
X_with_const = np.hstack([np.ones((X.shape[0], 1)), X])

## Benchmark regressions

### Baseline: normal equation

In [4]:
coefs = np.linalg.solve(X_with_const.T @ X_with_const, X_with_const.T @ y)
r2 = pearsonr(X_with_const @ coefs, y)[0] ** 2

print(f"""Linear regression results from the normal equation
* coefs = {coefs}
* R^2 = {r2}""")

Linear regression results from the normal equation
* coefs = [ 3.64911033e+01 -1.07170557e-01  4.63952195e-02  2.08602395e-02
  2.68856140e+00 -1.77957587e+01  3.80475246e+00  7.51061703e-04
 -1.47575880e+00  3.05655038e-01 -1.23293463e-02 -9.53463555e-01
  9.39251272e-03 -5.25466633e-01]
* R^2 = 0.7406077428649426


In [5]:
%%timeit -n 1000
coefs = np.linalg.solve(X_with_const.T @ X_with_const, X_with_const.T @ y)
r2 = pearsonr(X_with_const @ coefs, y)[0] ** 2

113 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Statsmodels OLS

In [6]:
lm_sm = sm.OLS(y, X_with_const).fit()
print(lm_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Fri, 03 May 2019   Prob (F-statistic):          6.95e-135
Time:                        15:22:44   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.4911      5.104      7.149      0.0

In [7]:
%%timeit -n 1000
lm_sm = sm.OLS(y, X_with_const).fit()

613 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Scikit-learn linear model


In [8]:
sk_ols = LinearRegression(fit_intercept=False)

In [9]:
lm_sk = sk_ols.fit(X_with_const, y)
score_sk = lm_sk.score(X_with_const, y)  # this calculates the R^2
print(f"""Scikit-learn linear regression results
* coefs = {lm_sk.coef_}
* R^2 = {score_sk}""")

Scikit-learn linear regression results
* coefs = [ 3.64911033e+01 -1.07170557e-01  4.63952195e-02  2.08602395e-02
  2.68856140e+00 -1.77957587e+01  3.80475246e+00  7.51061703e-04
 -1.47575880e+00  3.05655038e-01 -1.23293463e-02 -9.53463555e-01
  9.39251272e-03 -5.25466633e-01]
* R^2 = 0.7406077428649428


In [10]:
%%timeit -n 500
lm_sk = sk_ols.fit(X_with_const, y)
score_sk = lm_sk.score(X_with_const, y)

642 µs ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


## Summary

1. Speed ranking: normal equation > Scikit-learn `LinearRegression` ≈ Statsmodels `OLS`.
2. The Statsmodels `OLS` class provides a rich set of statistics for diagnosing the goodness of the fit, which does not exist in the results from other methods.
3. Given that there is not a significant difference between the performance of Scikit-learn `LinearRegression` and that of Statsmodels `OLS`, **Statsmodels should be the preferred package for performing linear regression in Python**.
4. For bootstrapping when the performance of a single iteration is critical, the normal equation may be preferred.